# CitiBike TSP
**Algorithms Group 50**\
Donal Lowsley-Williams \
Joan La Rosa \
Isaac Lichter \
Weidong Gao 

We will be conducting the traveling salesman problem on New York City citibike data. 

### Imports
Will we be implementing these algorithms by hand, so we just need libraries for data loading and export, matrix manipulation functions, and some data exploration. 

In [2]:
import csv
import json
import numpy as np
import pandas as pd

# The Data
We will get the data from CitiBike's openly available [System Data](https://s3.amazonaws.com/tripdata/index.html). The data we are using was the most recent as of this notebook's creation date, and was published by CitiBike on [**Oct 4th 2021, 01:26:29 pm**](https://s3.amazonaws.com/tripdata/202109-citibike-tripdata.csv.zip). This is the data of all rides taken on the CitiBike system during the month of September 2021. However, the preprocessing should work for any selection of the data from the system data, since they all should follow the same structure. 

Please store the data in a folder named **"Data"** with the name **"citibike-data.csv"**, like so: **"Data/citibike-data.csv**

In [77]:
df = pd.read_csv('Data/citibike-data.csv',
                 parse_dates=['started_at','ended_at'])

## Preprocessing
In order to preprocess the data, 

### Data Cleaning

First lets take a look at the length of our data.

In [78]:
len(df)

3280221

Now, lets see what the column was that caused data type issues. We can see from the error above that it was column 7, which corresponds to the `end_station_id`. 

In [79]:
df.columns[7]

'end_station_id'

In the data, we can see by looking at the csv file that these values are typically float values, so we can cast them to a numeric value. We set any values that can't be cast to a numeric value to `NaN` by setting `errors='coerce'`. 

In [80]:
df['end_station_id'] = df['end_station_id'].apply(pd.to_numeric, errors='coerce')

We drop all null values. (those with `NaN`) This includes not just those with a faulty `end_station_id` value but those with errors elsewhere. We want to use only entries with full and complete data.

In [81]:
df = df.dropna()

We can see that we dropped approx. 15K entries that had data issues or invalid end stations. This is negligible in the grand scheme of the project, given that we have about 3.2M entries. 

In [82]:
len(df)

3264732

Let's take a peek at our data and its types. We can see that many use the dtype object, which pandas uses to refer to strings with varying length. We can leave this be. We have a couple date items, which we set when reading the csv file. The remaining data is all float data. 

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3264732 entries, 0 to 3280220
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    float64       
 6   end_station_name    object        
 7   end_station_id      float64       
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(6), object(5)
memory usage: 348.7+ MB


Let's conduct a quick sanity check to ensure the data is properly cleaned:

In [97]:
temp = df.copy()

In [98]:
temp['start_station_name'] = temp.start_station_name.str.replace(r'[^a-zA-Z0-9\s]',r'',regex=True)

In [99]:
print('start station id uniques: ',len(temp['start_station_id'].unique()))
print('start station name uniques: ',len(temp['start_station_name'].unique()))
print(len(temp['start_station_id'].unique()) == len(temp['start_station_name'].unique()))

start station id uniques:  1488
start station name uniques:  1493
False


In [100]:
print('start station id uniques: ',len(df['start_station_id'].unique()))
print('start station name uniques: ',len(df['start_station_name'].unique()))
print(len(df['start_station_id'].unique()) == len(df['start_station_name'].unique()))

start station id uniques:  1488
start station name uniques:  1493
False


In [101]:
new = temp[['start_station_id', 'start_station_name']].copy()

In [102]:
new.drop_duplicates().groupby('start_station_id').count().sort_values('start_station_name').tail(10)

,start_station_name
start_station_id,
5303.08,1
5303.06,1
5300.06,1
5300.05,1
8841.03,1
5382.07,2
4488.09,2
5190.09,2
5422.04,2


In [104]:
b= new.drop_duplicates().loc[new.drop_duplicates()['start_station_id'] == 5382.07]

In [105]:
b['start_station_name']

343988     Forsyth Stt Grand St
643885    Forsyth St\t Grand St
Name: start_station_name, dtype: object

In [95]:
np.array(b['start_station_name'])

array(['Forsyth Stt Grand St', 'Forsyth St\t Grand St'], dtype=object)

True

### Feature Engineering
Now we need to include some additional features in order to effectively run our algorithms. Specifically, we need to create a distance measure between each station, and store this in a distance matrix. 

In [142]:
len(df['end_station_id'].unique()) == len(df['end_station_name'].unique())

False

In [135]:
df['distance_traveled'] = ((df['ended_at'] - df['started_at']) / np.timedelta64(1, 's')).astype(np.int32)

# The Algorithms
Now that we have pulled and preprocessed our data, we can begin to test various algorithms against it. We will test the following algorithms:

**Greedy**
- Nearest Neighbor
- Farthest First Traversal

**?????**
- Christofides Algorithm 

**K-opt Heuristic Algorithms**
- 2-OPT
- 3-OPT
- Lin-Kernighan Heuristic

## Greedy Algorithms
First, we will examine various greedy algorithms and their effectiveness on the data. 

### Nearest Neighbor

The nearest neighbor algorithm is very simple and simply starts at a random node (or station in our example) and continues to visit unvisited nodes that are as close to the last visited node as possible .

**Psuedocode:** \
These are the steps of the algorithm:

    Initialize all vertices as unvisited.
    Select an arbitrary vertex, set it as the current vertex u. Mark u as visited.
    Find out the shortest edge connecting the current vertex u and an unvisited vertex v.
    Set v as the current vertex u. Mark v as visited.
    If all the vertices in the domain are visited, then terminate. Else, go to step 3.

The sequence of the visited vertices is the output of the algorithm. 

**Time complexity:** Worst Case O(N^2)

**Space complexity:** Worst Case O(N)

[Source (wikipedia)](https://en.wikipedia.org/wiki/Nearest_neighbour_algorithm)

In [ ]:
# Nearest Neighbor
def NearestNeighbor(station_ids : list, distance_matrix : list)
    cost = 0
    path = []
    
    return path, cost

### Farthest-First Traversal

**PseudoCode:**\
The farthest-first traversal of a finite point set may be computed by a greedy algorithm that maintains the distance of each point from the previously selected points, performing the following steps:[3]

    Initialize the sequence of selected points to the empty sequence, and the distances of each point to the selected points to infinity.
    While not all points have been selected, repeat the following steps:
        Scan the list of not-yet-selected points to find a point p that has maximum distance from the selected points.
        Remove p from the not-yet-selected points and add it to the end of the sequence of selected points.
        For each remaining not-yet-selected point q, replace the distance stored for q by the minimum of its old value and the distance from p to q.

**Time complexity:** Worst Case O(N^2)

**Space complexity:** Worst Case O(N)

[Source (wikipedia)](https://en.wikipedia.org/wiki/Farthest-first_traversal)

In [123]:
# Farthest-First Traversal